# Load packages

In [1]:
import numpy as np
import pandas as pd

# Load data

In [2]:
spread_df=[]
years_list = [18,17,16,15,14,13,12,11,10,'09','08','07','06','05','04','03']
year_int = 2019
for year in years_list:
    #Note: 18 refers to 2019 and so on
    temp_spread_df = pd.read_csv("Data/Point spreads/ncaabb" + str(year) + ".csv")
    temp_spread_df['Season'] = year_int
    year_last = year
    year_int = year_int-1    
    if year==18:
        spread_df = temp_spread_df
    else:
        spread_df = spread_df.append(temp_spread_df)

C:\Users\dlorenz\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
spread_df['home'] = spread_df['home'].str.lower()
spread_df['road'] = spread_df['road'].str.lower()

In [4]:
spread_df.head()

,Season,date,home,hscore,line,line7ot,lineargh,lineash,lineashby,lineavg,...,linesaggm,linesagp,linesauce,lineseven,linetalis,lineteamrnks,neutral,road,rscore,std
0,2019,11/6/2018,arizona st.,102,12.5,11.0,NaN,NaN,NaN,12.88,...,12.38,12.38,NaN,NaN,16.7,14.3,0.0,cal st. fullerton,94,2.32
1,2019,11/6/2018,auburn,101,20.5,17.0,NaN,NaN,NaN,19.70,...,18.22,18.22,NaN,NaN,21.4,16.0,0.0,south alabama,58,2.63
2,2019,11/6/2018,ball st.,86,7,2.0,NaN,NaN,NaN,4.65,...,2.22,2.22,NaN,NaN,3.1,7.1,0.0,indiana st.,69,3.42
3,2019,11/6/2018,boston college,73,16,7.0,NaN,NaN,NaN,15.76,...,15.91,15.91,NaN,NaN,16.8,15.8,0.0,wisconsin-milwaukee,53,3.53
4,2019,11/6/2018,buffalo,82,15.5,11.0,NaN,NaN,NaN,14.47,...,11.41,11.41,NaN,NaN,14.4,14.3,0.0,st. francis (pa),67,4.90


# Match Team IDs to data

In [5]:
teams_df = pd.read_csv('Data/Kaggle NCAA/TeamSpellings.csv', sep='\,', engine='python')

In [6]:
teams_df.head()

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101


In [7]:
team_list = ['home','road']
for team in team_list:
    spread_df = pd.merge(spread_df, teams_df, left_on=[team], right_on = ['TeamNameSpelling'], how='left')
    if team=='home':
        spread_df.rename(columns={'TeamID': 'HTeamID'}, inplace=True)
    if team=='road':
        spread_df.rename(columns={'TeamID': 'RTeamID'}, inplace=True)
    spread_df = spread_df.drop(['TeamNameSpelling'], axis=1)

Note: change to home and road team and score

In [8]:
spread_df.loc[spread_df['hscore']>spread_df['rscore'], 'WTeamID'] = spread_df['HTeamID']
spread_df.loc[spread_df['hscore']<spread_df['rscore'], 'LTeamID'] = spread_df['HTeamID']

spread_df.loc[spread_df['hscore']>spread_df['rscore'], 'WScore'] = spread_df['hscore']
spread_df.loc[spread_df['hscore']<spread_df['rscore'], 'LScore'] = spread_df['hscore']

In [9]:
spread_df.loc[spread_df['rscore']>spread_df['hscore'], 'WTeamID'] = spread_df['RTeamID']
spread_df.loc[spread_df['rscore']<spread_df['hscore'], 'LTeamID'] = spread_df['RTeamID']

spread_df.loc[spread_df['rscore']>spread_df['hscore'], 'WScore'] = spread_df['rscore']
spread_df.loc[spread_df['rscore']<spread_df['hscore'], 'LScore'] = spread_df['rscore']

In [10]:
spread_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['WTeamID','LTeamID','WScore','LScore','line'])
spread_df = spread_df[spread_df['WScore'] != "."]
spread_df = spread_df[spread_df['LScore'] != "."]
spread_df = spread_df[spread_df['line'] != "."]

In [11]:
spread_df['line'].value_counts(dropna=False)

3.0       1575
5.0       1552
3.5       1548
2.5       1531
4.0       1521
4.5       1521
5.5       1519
6.0       1457
2.0       1456
6.5       1425
7.5       1403
7.0       1388
8.0       1310
8.5       1235
9.0       1152
1.5       1145
-2.0      1110
1.0       1058
9.5       1034
-2.5      1016
10.0       994
-1.5       981
-1.0       978
-3.0       969
10.5       909
0.0        880
11.0       873
-3.5       825
11.5       798
-4.0       790
          ... 
-72.5        1
-67.5        1
-74.0        1
34.0         1
40.5         1
29.5         1
-79.5        1
-77.0        1
-28.5        1
-25.5        1
-146.0       1
-75.5        1
-66.0        1
-24.5        1
-64.5        1
-60.0        1
-70.0        1
-34.0        1
46.0         1
33.5         1
42.5         1
40.5         1
39.5         1
-25.0        1
-66.0        1
15.54        1
26.5         1
-69.0        1
-23.0        1
-20.0        1
Name: line, Length: 268, dtype: int64

In [12]:
#drop spreads with bad coverage
spread_df = spread_df.drop(['line7ot','lineargh','lineash','lineashby','linedd','linedunk','lineer','linegreen','linemarkov','linemass','linepib','linepig','linepir','linepiw','linepom','lineprophet','linerpi','lineround','linesauce','lineseven','neutral','lineteamrnks','linetalis','lineespn','linemassey','linedonc','linesaggm','std','linepugh','linefox','linedok','lineopen'], axis=1)

In [13]:
spread_df.tail()

,Season,date,home,hscore,line,lineavg,linemoore,linesag,linesage,linesagp,road,rscore,HTeamID,RTeamID,WTeamID,LTeamID,WScore,LScore
58803,2004,03/30/2004,iowa st.,81,2.0,1.22,1.61,1.50,1.25,1.74,rutgers,84,1235.0,1353.0,1353.0,1235.0,84,81
58804,2004,04/01/2004,michigan,62,5.5,2.06,4.47,1.51,0.46,2.44,rutgers,55,1276.0,1353.0,1276.0,1353.0,62,55
58805,2004,04/03/2004,duke,78,-2.0,2.85,1.43,2.85,2.24,3.94,connecticut,79,1181.0,1163.0,1163.0,1181.0,79,78
58806,2004,04/03/2004,oklahoma st.,67,4.5,2.05,3.08,1.99,3.58,0.52,georgia tech,67,1329.0,1210.0,NaN,NaN,NaN,NaN
58807,2004,04/05/2004,connecticut,82,5.0,1.65,4.57,1.12,1.31,0.91,georgia tech,73,1163.0,1210.0,1163.0,1210.0,82,73


In [14]:
spread_df.dropna(subset=['line'])

,Season,date,home,hscore,line,lineavg,linemoore,linesag,linesage,linesagp,road,rscore,HTeamID,RTeamID,WTeamID,LTeamID,WScore,LScore
0,2019,11/6/2018,arizona st.,102,12.5,12.88,7.91,12.38,12.38,12.38,cal st. fullerton,94,1113.0,1168.0,1113.0,1168.0,102,94
1,2019,11/6/2018,auburn,101,20.5,19.70,20.73,18.22,18.22,18.22,south alabama,58,1120.0,1375.0,1120.0,1375.0,101,58
2,2019,11/6/2018,ball st.,86,7,4.65,11.15,2.22,2.22,2.22,indiana st.,69,1123.0,1232.0,1123.0,1232.0,86,69
3,2019,11/6/2018,boston college,73,16,15.76,26.57,15.91,15.91,15.91,wisconsin-milwaukee,53,1130.0,1454.0,1130.0,1454.0,73,53
4,2019,11/6/2018,buffalo,82,15.5,14.47,10.59,11.41,11.41,11.41,st. francis (pa),67,1138.0,1384.0,1138.0,1384.0,82,67
5,2019,11/6/2018,cs northridge,84,-14.5,-14.17,-17.93,-13.70,-13.70,-13.70,new mexico,87,1169.0,1307.0,1307.0,1169.0,87,84
6,2019,11/6/2018,central florida,74,7,8.87,0.49,15.46,15.46,15.46,rider,70,1416.0,1351.0,1416.0,1351.0,74,70
7,2019,11/6/2018,clemson,100,27.5,26.87,29.82,24.10,24.10,24.10,citadel,80,1155.0,1154.0,1155.0,1154.0,100,80
9,2019,11/6/2018,davidson,83,18.5,19.62,28.61,16.83,16.83,16.83,cleveland st.,63,1172.0,1156.0,1172.0,1156.0,83,63
11,2019,11/6/2018,george mason,71,5,0.08,-2.01,5.20,5.20,5.20,pennsylvania,72,1206.0,1335.0,1335.0,1206.0,72,71


# Write the data to a csv

In [15]:
spread_df.to_csv('Data/~Created data/spreads_all.csv', index=False)